In [ ]:
# Most recent working: As of July 20, 2023

import difflib
import json
import dash
import dash_bootstrap_components as dbc
from flask import Flask
import dash_table
import pandas as pd

# read json files
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def print_dicts_css(dict1, dict2):
    def recursive_print(dict1, dict2, prefix=""):
        rows = []
        if isinstance(dict1, dict) and isinstance(dict2, dict):
            keys_sorted = sorted(dict1, key=lambda k: list(dict1.keys()).index(k))
            for key in keys_sorted:
                rows += recursive_print(dict1.get(key, "-"), dict2.get(key, "-"), prefix=prefix + str(key) + ": ")
        elif isinstance(dict1, list) and isinstance(dict2, list):
            for idx, (val1, val2) in enumerate(zip(dict1, dict2)):
                rows += recursive_print(val1, val2, prefix=prefix + f"[{idx}]: ")
        else:
            is_same = dict1 == dict2
            rows.append({"Path": f"{prefix}", "Ground Truth": str(dict1), "LLM Result": str(dict2), "Is Same": is_same})
        return rows
    
    return recursive_print(dict1, dict2)

# load the JSONs
json1 = read_json('example1.json')
json2 = read_json('example3.json')

# compare the JSONs
rows = print_dicts_css(json1, json2)

# convert to DataFrame for easier calculations
df = pd.DataFrame(rows)
df = df.rename(columns={'Dictionary 1': 'Ground Truth'})
df = df.rename(columns={'Dictionary 2': 'LLM Result'})
df['Path'] = df['Path'].str.replace(r'\[0\]:', '', regex=True)
# calculate % true
perc_true = ((df["Is Same"].sum() - 1) / (df.shape[0]-1)) * 100

server = Flask(__name__)
app = dash.Dash(server=server, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define conditional highlighting for the "LLM Result" column when it doesn't match the "Ground Truth" value
style_data_conditional = [
    {
        'if': {
            'filter_query': '{LLM Result} ne {Ground Truth}',
            'column_id': 'LLM Result'
        },
        'backgroundColor': '#F7FE2E',
        'color': 'black'
    }
]

# output the comparison result
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            dbc.Alert(f"Percent accuracy: {perc_true:.2f}%", color="info"),
            dash_table.DataTable(
                id='table',
                columns=[{"name": i, "id": i} for i in ["Path", "Ground Truth", "LLM Result", "Is Same"]],
                data=df.to_dict('records'),
                style_data_conditional=style_data_conditional,
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold'
                },
                style_cell={
                    'whiteSpace': 'normal',
                    'height': 'auto',
                }
            )
        ])
    ])
])

if __name__ == '__main__':
    app.run_server(debug=True)
